In [169]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR  #
# from torch.utils.data import TensorDataset, \
#     DataLoader  # Own stuff set of data quality checks, tensor shapes might be different dataloader loads the tensor,
import pytorch_lightning as pl  #
from pytorch_lightning.callbacks.early_stopping import \
    EarlyStopping  # early stop when you reach optimum loss, 3 times in a row gradient descent
from pytorch_lightning.callbacks import LearningRateMonitor  # delta (loss / accuracy)
from pytorch_lightning.loggers import MLFlowLogger  # Model tracking
import torch
import torch.nn as nn
from transformers import DistilBertModel
from data_preprocessing import injestDataset, downSample
from data_cleaning import feature_cleaner, cleaningPreprocess
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer, LabelBinarizer
from sklearn.compose import ColumnTransformer

import torchmetrics
import pytorch_lightning as pl
from transformers import BertTokenizer
import numpy as np
# from transformers import DistilBertModel

In [170]:
torch.set_default_dtype(torch.float32)
# torch.set_default_device('mps:0')

In [171]:
data = injestDataset()

In [172]:
df = downSample(data,"Recommended IND",1000)

In [173]:
df.dtypes

trueIndex           int64
Review Text        object
Rating              int64
Recommended IND     int64
dtype: object

In [174]:
df = cleaningPreprocess(df, "Review Text")

In [175]:
df = df.rename(columns={"Recommended IND": "targetOne", "Review Text": "feature", "Rating": "targetTwo"})

In [176]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [177]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [178]:
# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, df):
        self.input_data = df
        # self.max_length = 30
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenize_dataframe()
        self.train_val_test_idx()
        self.data_split = None
        self.split_columns()
        self.targertOne_labelEncoder()
        self.targertTwo_labelEncoder()

    def tokenize_dataframe(self):
        tokenized_texts = []
        input_ids = []
        attention_masks = []

        for text in self.input_data['feature']:
            encoded_inputs = self.tokenizer(
                text,
                add_special_tokens=True,
                # max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            tokenized_texts.append(encoded_inputs)
            input_ids.append(encoded_inputs['input_ids'])
            attention_masks.append(encoded_inputs['attention_mask'])
        
        self.input_data['tokenizedFeature'] = tokenized_texts
        self.input_data['input_ids'] = input_ids
        self.input_data['attention_mask'] = attention_masks


    def split_columns(self):
        self.features = self.input_data["feature"]
        self.targetOne = self.input_data["targetOne"]
        self.targetTwo = self.input_data["targetTwo"]

            
    def train_val_test_idx(self):
        idx = list(range(len(self.input_data)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.input_data["targetOne"],
                                random_state=500)
                                
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.5,
                                random_state=500)

        self.train_idx = train_idx 
        self.test_idx = test_idx
        self.val_idx = val_idx,
    
    def targertOne_labelEncoder(self):
        self.targetOne_label_binarizer = MultiLabelBinarizer()
        transformed =  self.targetOne_label_binarizer.fit_transform(self.input_data["targetOne"].astype("str"))
        self.input_data['targetOne_binirized'] = [subarray[0] for subarray in np.split(transformed,len(transformed))]

    def targertTwo_labelEncoder(self):
        self.targetTwo_label_binarizer = MultiLabelBinarizer()
        transformed =  self.targetTwo_label_binarizer.fit_transform(self.input_data["targetTwo"].astype("str"))
        self.input_data['targetTwo_binirized'] = [subarray[0] for subarray in np.split(transformed,len(transformed))]

        
    def __len__(self):
        if self.data_split is None or self.data_split == 'all':
            # Return the length of the entire dataset
            return len(self.input_data)
        elif self.data_split == 'train':
            # Return the length of the training split
            return len(self.train_idx)
        elif self.data_split == 'val':
            # Return the length of the validation split
            return len(self.val_idx)
        elif self.data_split == 'test':
            # Return the length of the test split
            return len(self.test_idx)
        else:
            raise ValueError("Invalid data_split argument. Use 'train', 'val', 'test', or 'all'.")
        
    ## The whole purpose of this __getitem__ function is to return a dataframe or dataloader compatible data 

    def __getitem__(self, index):
        if self.data_split is not None:
            # Depending on the data_split argument, return the corresponding split
            if self.data_split == 'train':
                index = self.train_idx[index]
            elif self.data_split == 'val':
                index = self.val_idx[index]
            elif self.data_split == 'test':
                index = self.test_idx[index]
            else:
                raise ValueError("Invalid data_split argument. Use 'train', 'val', or 'test'.")

            input_ids = self.input_data['input_ids'][index]
            attention_mask = self.input_data['attention_mask'][index]
            targetOne = torch.tensor(self.input_data['targetOne_binirized'][index])
            targetTwo = torch.tensor(self.input_data['targetTwo_binirized'][index])

            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'targetOne': targetOne,
                'targetTwo': targetTwo,
            }

        else: 
            return self.input_data.iloc[index]

       # Define properties to access different data splits


    # def set_data_split(self, data_split):
    #     # This method allows you to set the data_split after creating the instance
    #     self.data_split = data_split

    # def reset_data_split(self):
    #     # This method allows you to reset the data_split to None, returning the entire DataFrame
    #     self.data_split = None

    @property
    def train(self):
        self.data_split = 'train'
        return self

    @property
    def val(self):
        self.data_split = 'val'
        return self

    @property
    def test(self):
        self.data_split = 'test'
        return self

    @property
    def all(self):
        self.data_split = None
        return self



In [179]:
# class MyDataset(Dataset):
#     def __init__(self, df, transform=None, data_split=None):
#         self.input_data = df
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         self.data_split = data_split  # Set the data_split attribute

#         self.tokenize_dataframe()
#         self.split_columns()
#         self.targertOne_labelEncoder()
#         self.targertTwo_labelEncoder()

#     # ... (rest of your dataset class remains the same) ...

#     def __getitem__(self, index):
#         if self.data_split is not None:
#             # Depending on the data_split argument, return the corresponding split
#             if self.data_split == 'train':
#                 index = self.train_idx[index]
#             elif self.data_split == 'val':
#                 index = self.val_idx[index]
#             elif self.data_split == 'test':
#                 index = self.test_idx[index]
#             else:
#                 raise ValueError("Invalid data_split argument. Use 'train', 'val', 'test', or None.")

#         input_ids = self.input_data['input_ids'][index]
#         attention_mask = self.input_data['attention_mask'][index]
#         targetOne = torch.tensor(self.input_data['targetOne_binirized'][index])
#         targetTwo = torch.tensor(self.input_data['targetTwo_binirized'][index])

#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'targetOne': targetOne,
#             'targetTwo': targetTwo,
#         }

#     def set_data_split(self, data_split):
#         # This method allows you to set the data_split after creating the instance
#         self.data_split = data_split

#     def reset_data_split(self):
#         # This method allows you to reset the data_split to None, returning the entire DataFrame
#         self.data_split = None


In [180]:
        # if data_split is None:
        #     # If data_split is not specified, return the entire row from the DataFrame
        #     sample = self.input_data.iloc[index]
        # else:
        #     # Depending on the data_split argument, return the corresponding split
        #     if data_split == 'train':
        #         index = self.train_idx[index]
        #     elif data_split == 'val':
        #         index = self.val_idx[index]
        #     elif data_split == 'test':
        #         index = self.test_idx[index]
        #     else:
        #         raise ValueError("Invalid data_split argument. Use 'train', 'val', or 'test'.")

        #     # Return the data for the specified split
        #     input_ids = self.input_data['input_ids'][index]
        #     attention_mask = self.input_data['attention_mask'][index]
        #     targetOne = torch.tensor(self.input_data['targetOne_binirized'][index])
        #     targetTwo = torch.tensor(self.input_data['targetTwo_binirized'][index])

        #     sample = {
        #         'input_ids': input_ids,
        #         'attention_mask': attention_mask,
        #         'targetOne': targetOne,
        #         'targetTwo': targetTwo,
        #     }

        # return sample

In [181]:
dataset = MyDataset(df)

In [182]:
dataset.data_split 

In [183]:
len(dataset)

1000

In [184]:
train = dataset.train

In [185]:
len(train)


800

In [186]:
test = dataset.test

In [187]:
len(test)

100

In [191]:
val = dataset.val

In [192]:
len(val)

1

In [ ]:
# dataset[1:6]["attention_mask"][1]

In [ ]:
# dataset[1:6]["attention_mask"][1]

In [ ]:
# dataset.targetTwo_label_binarizer.inverse_transform(np.array([[0, 0, 0, 1, 0]]))

In [ ]:
# def dataLoader():
#     trainDataLoader = DataLoader(dataset[dataset.train_idx], batch_size=10, shuffle=True)
#     testDataLoader = DataLoader(dataset[dataset.test_idx], batch_size=10, shuffle=True)
#     valDataLoader = DataLoader(dataset[dataset.val_idx], batch_size=10, shuffle=True)
#     return trainDataLoader, testDataLoader, valDataLoader

In [ ]:
# class Embedding(nn.Module):
#    def __init__(self):
#        super(Embedding, self).__init__()
#        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
#        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
#        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
#        self.norm = nn.LayerNorm(d_model)

#    def forward(self, x, seg):
#        seq_len = x.size(1)
#        pos = torch.arange(seq_len, dtype=torch.long)
#        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
#        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
#        return self.norm(embedding)

In [ ]:
x, y, z = dataLoader()

In [ ]:
dataset[[1]]

,trueIndex,feature,targetTwo,targetOne,tokenizedFeature,input_ids,attention_mask,targetOne_binirized,targetTwo_binirized
1,4578,great spring time dress looks good thin top ca...,5,1,"[input_ids, token_type_ids, attention_mask]","[[tensor(101), tensor(2307), tensor(3500), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ...","[0, 1]","[0, 0, 0, 0, 1]"


In [ ]:
class MultiClassClassifier(pl.LightningModule):
    def __init__(self, output_dim=4, input_dim=512, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
        # lower than epoch higher than iterations
        super().__init__()
        self.save_hyperparameters()  # save_hyperparameters
        # pl.seed_everything(seed)
        self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.pre_classifier = torch.nn.Linear(512, 512)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(512,4)

    def forward(self, batch):
        # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
        # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
        embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
        pooler = self.pre_classifier(embeddings)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("test_loss", loss)

    def predict(self, x):
        # TODO this will be changed
        self.eval()
        logits = self.forward(x)
        self.train()
        return torch.argmax(logits, dim=1).detach().numpy()

In [ ]:
model = MultiClassClassifier(x)

In [ ]:
# trainer = pl.Trainer(max_epochs=1,accelerator="mps")
# trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name          

MisconfigurationException: `train_dataloader` must be implemented to be used with the Lightning Trainer

In [ ]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader

In [ ]:
# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         # lower than epoch higher than iterations
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         # self.pre_classifier = torch.nn.Linear(768, 768)
#         # self.dropout = torch.nn.Dropout(0.3)
#         # self.classifier = torch.nn.Linear(768,output_dim)

#     def forward(self, batch):
#         # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
#         # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
#         return [optimizer], [scheduler]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         # TODO this will be changed
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().numpy()

#     # def df_to_tensor(self, df, target_col=None, format=np.float32):
#     #     if target_col is not None:
#     #         feature_cols = [col for col in df.columns if col != target_col]
#     #         tensor = TensorDataset(
#     #             torch.tensor(df[feature_cols].values.astype(format)),
#     #             torch.tensor(df[target_col].values),
#     #         )
#     #     else:
#     #         tensor = torch.tensor(df.values.astype(format))
#     #     return tensor

#     def tensor_to_loader(self, tensor, batch_size, num_workers, shuffle=True):
#         loader = DataLoader(dataset=tensor, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
#         return loader

#     def setup_trainer(self, experiment, run_id, max_epochs):
#         # mlf_logger = MLFlowLogger(experiment_name=experiment)
#         # mlf_logger._run_id = run_id
#         trainer = pl.Trainer(
#             max_epochs=max_epochs,
#             callbacks=[EarlyStopping(monitor="val_loss"), LearningRateMonitor(logging_interval="epoch")],
#             # logger=mlf_logger,
#         )

#         return trainer

In [ ]:
# class CustomBertClassifier(nn.Module):
#     def __init__(self, bert_model):
#         super(CustomBertClassifier, self).__init__()
#         self.bert = bert_model
#         self.fc = nn.Linear(bert_model.config.hidden_size, 2)  # Binary classification

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids, attention_mask=attention_mask)
#         pooled_output = outputs['pooler_output']
#         logits = self.fc(pooled_output)
#         return logits

In [ ]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader